# Part 2: Feature Extraction & Classification
## Import libraries and set paths

In [1]:
import mne
import pandas as pd
import numpy as np
import os
import glob  # find files by pattern
import matplotlib.pyplot as plt

# Main project folder
base_path = r"E:\Personal Project\Recognizing Mood Disorder Using EEG"

# Folder with cleaned epoch files (*.fif)
input_folder = os.path.join(base_path, "filtered_EEG_data")

# Folder where we save the final CSV
output_path = base_path

print(f"Base path set to: {base_path}")
print(f"Input folder set to: {input_folder}")

# EEG frequency bands
BANDS = {
    "delta": [1.0, 4.0],
    "theta": [4.0, 8.0],
    "alpha": [8.0, 13.0],
    "beta":  [13.0, 30.0],
    "gamma": [30.0, 40.0]  # stop at 40 Hz (filtered data)
}

Base path set to: E:\Personal Project\Recognizing Mood Disorder Using EEG
Input folder set to: E:\Personal Project\Recognizing Mood Disorder Using EEG\filtered_EEG_data


## Load Subject Information

In [2]:
import os
import glob
import re

# Path to filtered EEG files
data_folder = r"E:\Personal Project\Recognizing Mood Disorder Using EEG\EEG Data\filtered_EEG_data"

# Get all *-epo.fif files
file_paths = glob.glob(os.path.join(data_folder, "*-epo.fif"))

print(f"Found {len(file_paths)} EEG files:\n")
for fp in file_paths[:5]:   # show first 5 files
    print(" -", os.path.basename(fp))

# Function to extract subject ID + label from filename
def parse_filename(file_path):
    name = os.path.basename(file_path)

    # Example filename:
    #   02010002_label_MDD-epo.fif
    # subject_id = 02010002
    # label      = MDD

    # extract first 8 digits
    id_match = re.search(r"(\d{8})", name)
    subject_id = id_match.group(1) if id_match else "UNKNOWN"

    # extract label after _label_
    label_match = re.search(r"_label_([A-Za-z]+)", name)
    label = label_match.group(1) if label_match else "UNKNOWN"

    return subject_id, label

# Quick test on few files
print("\nTesting parsing:")
for fp in file_paths[:3]:
    print(os.path.basename(fp), "->", parse_filename(fp))

Found 53 EEG files:

 - 02010002_label_MDD-epo.fif
 - 02010004_label_MDD-epo.fif
 - 02010005_label_MDD-epo.fif
 - 02010006_label_MDD-epo.fif
 - 02010008_label_MDD-epo.fif

Testing parsing:
02010002_label_MDD-epo.fif -> ('02010002', 'MDD')
02010004_label_MDD-epo.fif -> ('02010004', 'MDD')
02010005_label_MDD-epo.fif -> ('02010005', 'MDD')


## Apply HydroCel-128 Montage

In [3]:
# Load official HydroCel-128 montage
montage = mne.channels.make_standard_montage("GSN-HydroCel-128")

# Load one sample epochs file
temp_epochs = mne.read_epochs(file_paths[0], preload=True, verbose="ERROR")

# Assign montage (adds 3D electrode positions)
temp_epochs.set_montage(montage)

# Print first 10 channels to verify
print("Channel info after applying montage:")
for ch in temp_epochs.info["chs"][:10]:
    print(f"{ch['ch_name']} | XYZ: {ch['loc'][:3]}")

Channel info after applying montage:
E1 | XYZ: [0.06266342 0.05976723 0.00143223]
E2 | XYZ: [0.05729458 0.07563959 0.0312517 ]
E3 | XYZ: [0.04183701 0.08854471 0.06000893]
E4 | XYZ: [0.03106102 0.08532577 0.08123536]
E5 | XYZ: [0.01601688 0.07154467 0.10243338]
E6 | XYZ: [0.         0.05243744 0.11605322]
E7 | XYZ: [-0.01325013  0.02880136  0.12436175]
E8 | XYZ: [0.0457107  0.08776722 0.01196957]
E9 | XYZ: [0.02918326 0.09991306 0.03735032]
E10 | XYZ: [0.01982304 0.10025007 0.06015118]


## Define EEG Regions from Channel Positions

In [4]:
import numpy as np

def get_regions(epochs):
    """Return dict: region_name -> list of channel names."""
    ch_names = epochs.ch_names
    pos = np.array([ch["loc"][:3] for ch in epochs.info["chs"]])  # (n_channels, 3)

    regions = {
        "frontal_left": [],
        "frontal_right": [],
        "central_left": [],
        "central_right": [],
        "parietal_left": [],
        "parietal_right": [],
        "occipital": []
    }

    for name, (x, y, z) in zip(ch_names, pos):
        # y: front (+) to back (–), x: left (–) to right (+)
        if y > 0.4:  # frontal
            if x < 0:
                regions["frontal_left"].append(name)
            else:
                regions["frontal_right"].append(name)
        elif 0.1 < y <= 0.4:  # central
            if x < 0:
                regions["central_left"].append(name)
            else:
                regions["central_right"].append(name)
        elif -0.3 < y <= 0.1:  # parietal
            if x < 0:
                regions["parietal_left"].append(name)
            else:
                regions["parietal_right"].append(name)
        else:  # y <= -0.3 -> occipital
            regions["occipital"].append(name)

    return regions

## Connectivity Setup

In [5]:
from scipy.signal import coherence  # compute coherence between signals

# Selected region pairs for coherence (reduced, meaningful set)
CONNECTIVITY_PAIRS = [
    ("frontal_left",  "frontal_right"),   # left–right frontal
    ("frontal_left",  "parietal_left"),   # frontal–parietal (left)
    ("frontal_right", "parietal_right"),  # frontal–parietal (right)
    ("frontal_left",  "occipital"),       # frontal–occipital (left)
    ("frontal_right", "occipital"),       # frontal–occipital (right)
]


## Feature Extraction (Power, Asymmetry, Coherence)

In [6]:
def extract_features(file_path, bands_dict, montage):
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    subject_id, label = parse_filename(file_path)

    # Load EEG epochs and apply montage
    epochs = mne.read_epochs(file_path, preload=True, verbose="ERROR")
    epochs.set_montage(montage)

    data = epochs.get_data()  # (epochs, channels, times)
    sfreq = epochs.info["sfreq"]
    ch_names = epochs.ch_names
    n_epochs, n_channels, n_times = data.shape

    print(f"  data shape: {data.shape}, sfreq: {sfreq}")

    # ---------------------------------------------------------
    # PSD (Welch) for band power
    # ---------------------------------------------------------
    psd_obj = epochs.compute_psd(
        method="welch",
        fmin=0.0,
        fmax=sfreq / 2,
        n_fft=2 * int(sfreq),
        n_overlap=int(0.5 * sfreq),
        verbose=False
    )
    psd, freqs = psd_obj.get_data(return_freqs=True)

    # Build anatomical regions
    regions = get_regions(epochs)

    # Store all features
    features = {"subject_id": subject_id, "label": label}
    eps = 1e-12

    # ---------------------------------------------------------
    # 1) Absolute + Relative Power per Region
    # ---------------------------------------------------------
    for region_name, region_channels in regions.items():
        if len(region_channels) == 0:
            continue

        idx = [ch_names.index(ch) for ch in region_channels]

        total_mask = (freqs >= 1) & (freqs <= 40)
        total_power = psd[:, idx, :][:, :, total_mask].sum(axis=-1).mean()

        for band_name, (fmin, fmax) in bands_dict.items():
            band_mask = (freqs >= fmin) & (freqs < fmax)
            if not np.any(band_mask):
                continue

            band_power = psd[:, idx, :][:, :, band_mask].sum(axis=-1).mean()

            features[f"{band_name}_abs_{region_name}"] = band_power
            features[f"{band_name}_rel_{region_name}"] = band_power / (total_power + eps)

    # ---------------------------------------------------------
    # 2) Frontal Alpha Asymmetry (FAA)
    # ---------------------------------------------------------
    try:
        L = features["alpha_abs_frontal_left"]
        R = features["alpha_abs_frontal_right"]
        features["alpha_FAA_frontal"] = np.log(R + eps) - np.log(L + eps)
    except:
        pass  # skip if region missing

    # ---------------------------------------------------------
    # 3) Alpha-Band Coherence (Scipy)
    # ---------------------------------------------------------
    # Create region-average signals
    region_ts = {}
    for region_name, region_channels in regions.items():
        if len(region_channels) == 0:
            continue
        idx = [ch_names.index(ch) for ch in region_channels]
        region_ts[region_name] = data[:, idx, :].mean(axis=1)

    # Compute coherence for selected pairs
    for (A, B) in CONNECTIVITY_PAIRS:
        if A not in region_ts or B not in region_ts:
            continue

        coh_vals = []
        for ep in range(n_epochs):
            f, Cxy = coherence(
                region_ts[A][ep],
                region_ts[B][ep],
                fs=sfreq,
                nperseg=int(sfreq * 2)
            )
            alpha_mask = (f >= 8) & (f <= 13)
            if np.any(alpha_mask):
                coh_vals.append(np.mean(Cxy[alpha_mask]))

        if len(coh_vals) > 0:
            features[f"alpha_coh_{A}__{B}"] = float(np.mean(coh_vals))

    return features

## Run Feature Extraction Loop

In [7]:
all_features_list = []

print("\n=== Starting Feature Extraction (with connectivity) ===")

for file_path in file_paths:
    try:
        feat = extract_features(file_path, BANDS, montage)
        all_features_list.append(feat)
    except Exception as e:
        print("ERROR:", file_path, e)



=== Starting Feature Extraction (with connectivity) ===

Processing: 02010002_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010004_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010005_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010006_label_MDD-epo.fif
  data shape: (157, 128, 500), sfreq: 250.0

Processing: 02010008_label_MDD-epo.fif
  data shape: (165, 128, 500), sfreq: 250.0

Processing: 02010010_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010011_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010012_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010013_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 02010015_label_MDD-epo.fif
  data shape: (151, 128, 500), sfreq: 250.0

Processing: 02010016_label_MDD-epo.fif
  data shape: (150, 128, 500), sfreq: 250.0

Processing: 020100

## Create DataFrame and Save CSV

In [8]:
df = pd.DataFrame(all_features_list)
df.set_index("subject_id", inplace=True)

df.to_csv(
    r"E:\Personal Project\Recognizing Mood Disorder Using EEG\EEG Data\features_power_asym_connectivity_scipy.csv"
)

print("Saved:", df.shape)


Saved: (53, 41)


# Classification

## Load Extracted Features & ML Libraries

In [9]:
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Path to the saved features CSV
features_csv_path = r"E:\Personal Project\Recognizing Mood Disorder Using EEG\EEG Data\features_power_asym_connectivity_scipy.csv"

# Load feature table (subject_id is the index)
features_df = pd.read_csv(features_csv_path, index_col=0)

print("Loaded features:", features_df.shape)
print(features_df.head())

Loaded features: (53, 41)
           label  delta_abs_central_left  delta_rel_central_left  \
subject_id                                                         
2010002      MDD            4.977989e-11                0.382350   
2010004      MDD            5.325629e-10                0.827610   
2010005      MDD            5.849932e-11                0.544899   
2010006      MDD            3.288739e-11                0.321503   
2010008      MDD            5.968350e-11                0.616444   

            theta_abs_central_left  theta_rel_central_left  \
subject_id                                                   
2010002               1.170719e-11                0.089921   
2010004               4.252737e-11                0.066088   
2010005               6.725975e-12                0.062650   
2010006               4.477051e-12                0.043767   
2010008               6.089076e-12                0.062891   

            alpha_abs_central_left  alpha_rel_central_left  \


## Prepare Data (Clean, Split X & y)

In [10]:
# Remove any rows with missing values
features_df = features_df.dropna()
print("After dropna:", features_df.shape)

# Check that label column exists
if "label" not in features_df.columns:
    raise KeyError("Column 'label' not found.")

# Map labels to numbers
label_mapping = {"HC": 0, "MDD": 1}
y = features_df["label"].map(label_mapping)

# Drop rows with unmapped labels
if y.isna().any():
    print("Warning: Unmapped labels found. Dropping those rows:")
    print(features_df[y.isna()][["label"]])
    mask = ~y.isna()
    features_df = features_df[mask]
    y = y[mask]

# X = all features except label
X = features_df.drop(columns=["label"])

print("Final X shape:", X.shape)
print("Final y length:", len(y))
print("Label counts:\n", y.value_counts())

After dropna: (53, 41)
Final X shape: (53, 40)
Final y length: 53
Label counts:
 label
0    29
1    24
Name: count, dtype: int64


## Train/Test Split & Standardization

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split subjects (20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train X:", X_train.shape)
print("Test  X:", X_test.shape)

# Scale features (mean=0, std=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Scaling complete.")

Train X: (42, 40)
Test  X: (11, 40)
Scaling complete.


## Train Random Forest Classifier

In [12]:
rf_clf = RandomForestClassifier(
    n_estimators=300,        # number of trees
    random_state=42,
    class_weight="balanced", # handle class imbalance
    n_jobs=-1                # use all CPU cores
)

rf_clf.fit(X_train_scaled, y_train)
print("Random Forest training complete.")

Random Forest training complete.


## Evaluate Random Forest on Test Set

In [13]:
y_pred = rf_clf.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("\n=== Random Forest Results ===")
print(f"Accuracy: {acc:.3f}\n")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", report)


=== Random Forest Results ===
Accuracy: 0.364

Confusion Matrix:
 [[3 3]
 [4 1]]

Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.50      0.46         6
           1       0.25      0.20      0.22         5

    accuracy                           0.36        11
   macro avg       0.34      0.35      0.34        11
weighted avg       0.35      0.36      0.35        11



## Train SVM Classifier (RBF Kernel)

In [14]:
from sklearn.svm import SVC

svm_clf = SVC(
    kernel="rbf",            # non-linear kernel
    C=1.0,                   # regularization strength
    gamma="scale",           # auto gamma
    class_weight="balanced", # handle class imbalance
    random_state=42
)

svm_clf.fit(X_train_scaled, y_train)
print("SVM training complete.")

SVM training complete.


## Evaluate SVM on Test Set

In [15]:
y_pred_svm = svm_clf.predict(X_test_scaled)

acc_svm = accuracy_score(y_test, y_pred_svm)
cm_svm = confusion_matrix(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)

print("\n=== SVM (RBF) Results ===")
print(f"Accuracy: {acc_svm:.3f}\n")
print("Confusion Matrix:\n", cm_svm)
print("\nClassification Report:\n", report_svm)


=== SVM (RBF) Results ===
Accuracy: 0.455

Confusion Matrix:
 [[4 2]
 [4 1]]

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.67      0.57         6
           1       0.33      0.20      0.25         5

    accuracy                           0.45        11
   macro avg       0.42      0.43      0.41        11
weighted avg       0.42      0.45      0.43        11



## Compare Random Forest vs SVM

In [16]:
print(f"Random Forest accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"SVM (RBF) accuracy:     {acc_svm:.3f}")

Random Forest accuracy: 0.364
SVM (RBF) accuracy:     0.455
